In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_5_8_8,0.884494,0.677158,0.836619,0.789366,13.930106,29.614584,13.317816,21.945562,"Hidden Size=[30, 15], regularizer=0.02, learni..."
1,model_5_8_9,0.883970,0.664944,0.834928,0.783051,13.993230,30.735033,13.455622,22.603527,"Hidden Size=[30, 15], regularizer=0.02, learni..."
2,model_5_8_11,0.882297,0.642434,0.829037,0.770390,14.194999,32.799850,13.935793,23.922604,"Hidden Size=[30, 15], regularizer=0.02, learni..."
3,model_5_8_10,0.882141,0.648990,0.827968,0.773053,14.213799,32.198460,14.022974,23.645224,"Hidden Size=[30, 15], regularizer=0.02, learni..."
5,model_7_5_3,0.881501,0.751783,0.915550,0.855203,14.291005,18.265646,11.832665,15.238371,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
...,...,...,...,...,...,...,...,...,...,...
350,model_7_9_0,0.765038,0.731651,0.961833,0.819876,28.336430,26.353868,1.930031,14.860311,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
374,model_3_2_12,0.760381,0.459722,0.627691,0.560905,28.898075,49.330704,51.617363,50.406803,"Hidden Size=[15, 7], regularizer=0.1, learning..."
381,model_3_2_7,0.759577,0.579068,0.808436,0.713883,28.995123,38.433670,26.558628,32.845413,"Hidden Size=[15, 7], regularizer=0.1, learning..."
386,model_3_2_11,0.758922,0.455423,0.659668,0.577269,29.074034,49.723278,47.184021,48.528309,"Hidden Size=[15, 7], regularizer=0.1, learning..."
